# Retrive Metrics



Prerequisite :
1. A deployed prompt with monitoring enabled.(https://dataplatform.cloud.ibm.com/docs/content/wsj/model/wos-eval-prompt-spaces.html?context=wx&audience=wdp).
2. Your IBM Cloud API Key
3. The space id where your prompt is deployed.You can find it at Deployments > YOUR_SPACE > Manage

In [17]:
#install the required libraries
!pip install ibm-watson-openscale

In [18]:
#set the required variables
CLOUD_API_KEY = "YOUR_IBM_CLOUD_API_KEY"
SPACE_ID = "YOUR_SPACE_ID"

In [19]:
#setup the openscale client
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
client = APIClient(authenticator=authenticator)
print(client.version)

3.0.39


In [20]:
#list available datamarts
client.data_marts.show()

AIOSFASTPATH-BB92D5C7-DA77-4244-B87F-285C120DE28F,None,True,active,2024-01-08 03:02:00.245000+00:00,bb92d5c7-da77-4244-b87f-285c120de28f


### Select the data mart where your monitoring is happenning. You can get this information from the Evaluations Tab > Actions > View Model information

In [21]:
DATA_MART_ID = "bb92d5c7-da77-4244-b87f-285c120de28f"

In [22]:
#list subscriptions for the given deployment space
client.subscriptions.show(space_id=SPACE_ID)

398e6622-3aad-4123-8e4a-2a31ff837e44,prompt,Identify Negative news,bb92d5c7-da77-4244-b87f-285c120de28f,e4bc35de-97ca-4a76-a4f6-758f5be09a9e,Negative news classification production,8b5cca8a-8553-475f-8f6b-6a01fe77c6e5,active,2024-05-09 12:28:54.654000+00:00,b433fd9d-da02-42a7-a705-62291266389a
2ffb8c0b-c8cb-466f-af4b-637b894c2389,prompt,Summarize new articles,bb92d5c7-da77-4244-b87f-285c120de28f,53dd60e5-49e5-43d6-a782-935270b1a51e,Negative news summarization production,8b5cca8a-8553-475f-8f6b-6a01fe77c6e5,active,2024-05-07 15:39:04.484000+00:00,f02db6fb-39a7-4c56-8419-f2a5d85fd7ef


### Select the subscription where your monitoring is happenning. You can also get this information from the Evaluations Tab > Actions > View Model information

In [23]:
SUBSCRIPTION_ID = "f02db6fb-39a7-4c56-8419-f2a5d85fd7ef"

In [24]:
#get list of monitor instances available for your subscription
monitor_instances_info = client.monitor_instances.list(
        data_mart_id=DATA_MART_ID,
        target_target_id= SUBSCRIPTION_ID,
        target_target_type="subscription"
     )
print(monitor_instances_info.result.monitor_instances)

[<ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f4758c00550>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f4758c02980>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f4758c026e0>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f4758c02d70>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f4758c034f0>]


In [25]:
#list different monitor definitions available for your subscription
for instance in monitor_instances_info.result.monitor_instances:
    print(instance.entity.monitor_definition_id)

model_health
mrm
drift_v2
generative_ai_quality
sample_model_performance


In [26]:
def get_monitor_instance_details(monitor_definition):
    for instance in monitor_instances_info.result.monitor_instances:
        if instance.entity.monitor_definition_id == monitor_definition :
            return instance
    return -1

In [27]:
#get details of given monitor definitions. Pass any name listed in the above definition list 
instance_details = get_monitor_instance_details("model_health")
instance_id = instance_details.metadata.id
instance_id

'9a027540-a9e2-47b2-b0e7-d310034657cb'

In [28]:
#get metrics for the given time frame
#import datetime

#end_time = datetime.datetime.now()
#start_time = end_time - datetime.timedelta(days=1)
#print(start_time, end_time)
#metrics= client.monitor_instances.get_metrics(
#        monitor_instance_id=instance_id, 
#       start = start_time,
#       end= end_time
#)
#print(metrics.result)

2024-07-31 06:55:55.924736 2024-08-01 06:55:55.924736
{
  "start": "2024-07-31T06:00:00Z",
  "end": "2024-08-01T07:00:00Z",
  "interval": "hour",
  "monitor_definition_id": "model_health",
  "groups": []
}


In [29]:
#show metrics 
metrics= client.monitor_instances.show_metrics(
        monitor_instance_id=instance_id,  
)